**Libraries**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

***Board Games***

Return the main category on the game based on the input of the user :

In [ ]:
def main_category(): 
    
    url_categories = 'https://boardgamegeek.com/browse/boardgamecategory'
    soup_categories = BeautifulSoup(requests.get(url_categories).content)
    
    categories = soup_categories.find('table').find_all('a')
    list_categories = [cat.get_text() for cat in categories]
    
    print('List of categories available: ')
    print(list_categories)
    
    main_category = input('Which category do you want?')
    main_category = word_capitalize_all (main_category)
    
    if main_category not in list_categories:
        return 'The input category might seem to not exist. Maybe you have miss typed it? Try again.'
    else :
        return main_category

Return soup of the main category and the id og the games that belong to this main category :

In [ ]:
def soup(main_category):
    
    main_category = main_category.lower()
    
    url_board_game1 = 'https://boardgamegeek.com/xmlapi/search?search=' + main_category
    response_board_game1 = requests.get(url_board_game1)
    soup_board_game1 = BeautifulSoup(response_board_game1.content)
    
    if response_board_game1.status_code == 200:
        games = soup_board_game1.find_all('boardgame')
        list_game_id = [game.get('objectid') for game in games]
            
        if len(list_game_id) == 0:
            return 'It seems that no game are registered under this category'
        
        else :
            url_board_game2 = 'https://boardgamegeek.com/xmlapi/boardgame/'
            for id in list_game_id:
                url_board_game2 = url_board_game2 + id + ','
            url_board_game2 = url_board_game2.rstrip(',')
            url_board_game2 = url_board_game2 + '/?stats=1'
            
            response_board_game2 = requests.get(url_board_game2)
            soup_board_game2 = BeautifulSoup(response_board_game2.content)
            
            return soup_board_game2, list_game_id
    else :
        return 'A problem occured. Try again.'

Return a dataframe with all the games and features we want from this main category :

In [ ]:

def board_game_category (soup, game_id):
       
    all_game = soup.find_all('boardgame')
        
    my_features = {
        'game_id' : game_id,
        'game_name' : [game.find('name').get_text() for game in all_game],
        'game_year_released' : [int(game.find('yearpublished').get_text()) for game in all_game],
        'game_categories' : [[cat.text.strip() for cat in game.find_all('boardgamecategory')] for game in all_game],
        'game_min_player' : [int(game.find('minplayers').get_text()) for game in all_game],
        'game_max_player' : [int(game.find('maxplayers').get_text()) for game in all_game],
        'game_min_time' : [int(game.find('minplaytime').get_text()) for game in all_game],
        'game_max_time' : [int(game.find('maxplaytime').get_text()) for game in all_game],
        'game_age' : [int(game.find('age').get_text()) for game in all_game],
        'game_average' : [float(game.find('average').get_text()) for game in all_game],
        'game_gameplay' : [[play.text.strip() for play in game.find_all('boardgamemechanic')] for game in all_game],
        'game_description' : [game.find('description').get_text() for game in all_game]
    }
    
    df = pd.DataFrame(my_features)

    return df

Function where the player will have different input (criteria) that will give him a list of games at the end:

In [ ]:
def game_selector(game_dataset):
    
    print('Dear player, in order to get a list as short as possible, you are kindly ask to file as much information as possible. Please also note that the first 3 questions are mandatory.')
    
    players = int(input('How many players are you?'))
    game_dataset_1 = game_selector_players(game_dataset, players)
    
    age = int(input('How old are you?'))
    game_dataset_2 = game_selector_age(game_dataset_1, age)
    
    time = int(input('What it your limit of time?'))
    game_dataset_3 = game_selector_time(game_dataset_2, time).reset_index()
    
    if input('Do you want to add a second category? Yes or No').lower() == 'yes':
        print('You can see the most popular second category :')
        print(pd.DataFrame(list_second_category(game_dataset_3)))
        second_category = input('Which secondary category do you want? : ')
        game_dataset_4 = game_selector_second_category(game_dataset_3,second_category)
    else :
        game_dataset_4 = game_dataset_3
        
    if input('Do you want to add a minimum score? Yes or No').lower() == 'yes':
        average = input('What minimum score do you want?')
        game_dataset_5 = game_selector_average(game_dataset_4, float(average))
    else:
        game_dataset_5 = game_dataset_4
        
    if input('Do you want to add a year of publication? Yes or No').lower() == 'yes':
        year = input('What year do you want?')
        game_dataset_6 = game_selector_year(game_dataset_5, int(year))
    else:
        game_dataset_6 = game_dataset_5
        
    #if dataset empty, print('your are too strict with your inputs) or create a function that test if data set empty ?
    return game_dataset_6

Functions used for the function game_selector:

In [ ]:
def game_selector_players(df, number_players):  
    df_filtered = df[(df.game_min_player < number_players) & (df.game_max_player > number_players)]
    return df_filtered

In [ ]:
def game_selector_age(df, age_players):
    if age_players <= 18 :
        df_filtered = df[(df['game_age'] <= age_players)]
    else :
        df_filtered = df
    return df_filtered

In [ ]:
def game_selector_time(df, time_play):
    df_filtered = df[(df.game_max_time <= time_play)]
    return df_filtered

In [ ]:
def list_second_category(df):
    
    second_categoy_list_all = []
    second_category_cleaned = []
    
    elements = list(df.loc[:, 'game_categories'])
    for element in elements :
        for e in element:
            second_categoy_list_all.append(e)
        
    for category in second_categoy_list_all:
        if category not in second_category_cleaned:
            second_category_cleaned.append(category)

    return second_category_cleaned

In [ ]:
def game_selector_second_category(df, category):
    
    category_clean = word_capitalize_all (category)
    
    row_to_drop = []
    
    for i, row in enumerate(df.index):
        if not category_clean in df['game_categories'].iloc[i]:
            row_to_drop.append(i)
            
    df = df.drop(row_to_drop)
    
    return df

In [ ]:
def game_selector_average(df, average):
    df_filtered = df[(df.game_average > average)]
    return df_filtered

In [ ]:
def game_selector_year(df, year):
    df_filtered = df[(df.game_year_released > year-10) & (df.game_year_released < year+10)]
    return df_filtered

Function that is used at least two times and usefull to avoid repetition and make functions more clear:

In [ ]:
def word_capitalize_all (words_to_capitalize):
    words = words_to_capitalize.split() #creer a fonction wold splitter? useg again later
    cap = []
    for word in words:
        cap.append(word.capitalize())
    words_to_capitalize = " ".join(cap)
    return words_to_capitalize

**User interface :**

In [ ]:
main_cat = main_category()

In [ ]:
soup_cat, id_cat = soup(main_cat)[0], soup(main_cat)[1]

In [ ]:
board_game_reduced = board_game_category(soup_cat,id_cat)

In [ ]:
game_to_select = game_selector(board_game_reduced)

In [ ]:
game_to_select

***Video Games***

In [ ]:
df_videogames = pd.read_csv('VideogameDF_Cleaned.csv')

In [ ]:
df_videogames.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
df_videogames.head(5)

In [ ]:
def video_game_selector():
    
    game_mode = input("Choose a game mode: (Singeplayer, Multiplayer, Co-operative)")
    genre = input("Enter genre: (Adventure, Shooter, Role-playing (RPG), Platform, Simulator, Puzzle, Indie, Pinball, Sport, Racing)")
    platform = input("On what plaform do you want to play? (PC (Microsoft Windows), Android, iOS, PlayStation 5, Xbox One)")
    player_perspective = input("What player perspective should the game have? (Side view, Isometric, Third person, First person, Virtual Reality)")

    filtered_df = df_videogames[
    (df_videogames['game_modes'].apply(lambda x: game_mode in x)) &
    (df_videogames['genres'].apply(lambda x: genre in x)) &
    (df_videogames['platforms'].apply(lambda x: platform in x)) &
    (df_videogames['player_perspectives'].apply(lambda x: player_perspective in x))
    ]

    if filtered_df.empty:
        print("The perfect game doesn't exist, pick other criteria!")
    else:
        recommended_games_indices = random.sample(range(len(filtered_df)), min(5, len(filtered_df)))
        recommended_games_df = filtered_df.iloc[recommended_games_indices]
        print("\nThe gameseeker tells you to play one of the following games:")
        return recommended_games_df

Test this one with: Multiplayer, Shooter, PlayStation 5, First person

In [ ]:
video_game_selector()